In [1]:
import numpy as np
np.random.seed(1001)

import os
import shutil

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from tqdm import tqdm_notebook
from sklearn.cross_validation import StratifiedKFold

import librosa
import scipy
from keras import losses, models, optimizers
from keras.activations import relu, softmax
from keras.callbacks import (EarlyStopping, LearningRateScheduler,
                             ModelCheckpoint, TensorBoard, ReduceLROnPlateau)
from keras.layers import (Convolution1D, Dense, Dropout, GlobalAveragePooling1D, 
                          GlobalMaxPool1D, Input, MaxPool1D, concatenate)
from keras.utils import Sequence, to_categorical

from keras.layers import (Convolution2D, GlobalAveragePooling2D, BatchNormalization, Flatten,
                          GlobalMaxPool2D, MaxPool2D, concatenate, Activation)
from keras.utils import Sequence, to_categorical
from keras import backend as K
from keras.layers.advanced_activations import LeakyReLU
import kaggle_util
from util import Config, DataGenerator, audio_norm, prepare_data

DEBUG = 1

n_folds = 5
if DEBUG:
    n_folds = 2

/home/kownse/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/kownse/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def get_1d_dummy_model(config):
    
    nclass = config.n_classes
    input_length = config.audio_length
    
    inp = Input(shape=(input_length,1))
    x = GlobalMaxPool1D()(inp)
    out = Dense(nclass, activation=softmax)(x)

    model = models.Model(inputs=inp, outputs=out)
    opt = optimizers.Adam(config.learning_rate)

    model.compile(optimizer=opt, loss=losses.categorical_crossentropy, metrics=['acc'])
    return model

In [3]:
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/sample_submission.csv")

LABELS = list(train.label.unique())
label_idx = {label: i for i, label in enumerate(LABELS)}
train.set_index("fname", inplace=True)
test.set_index("fname", inplace=True)
train["label_idx"] = train.label.apply(lambda x: label_idx[x])
if DEBUG:
    train = train[:100]

In [4]:
config = Config(sampling_rate=16000, audio_duration=15, n_folds=n_folds, learning_rate=0.001)
if DEBUG:
    config = Config(sampling_rate=100, audio_duration=1, n_folds=n_folds, max_epochs=1)


PREDICTION_FOLDER = "predictions_1d_conv"
if not os.path.exists(PREDICTION_FOLDER):
    os.mkdir(PREDICTION_FOLDER)
if os.path.exists('logs/' + PREDICTION_FOLDER):
    shutil.rmtree('logs/' + PREDICTION_FOLDER)

skf = StratifiedKFold(train.label_idx, n_folds=config.n_folds)

for i, (train_split, val_split) in enumerate(skf):
    train_set = train.iloc[train_split]
    val_set = train.iloc[val_split]
    checkpoint = ModelCheckpoint('../model/best1d_%d.h5'%i, monitor='val_acc', verbose=1, save_best_only=True)
    early = EarlyStopping(monitor="val_loss", mode="min", patience=5)
    tb = TensorBoard(log_dir='./logs/' + PREDICTION_FOLDER + '/fold_%d'%i, write_graph=True)

    callbacks_list = [checkpoint, early, tb]
    print("Fold: ", i)
    print("#"*50)
    if not DEBUG:
        model = get_1d_conv_model_advance(config)
    else:
        model = get_1d_dummy_model(config)

    train_generator = DataGenerator(config, '../data/audio_train/', train_set.index, 
                                    train_set.label_idx, batch_size=64,
                                    preprocessing_fn=audio_norm)
    val_generator = DataGenerator(config, '../data/audio_train/', val_set.index, 
                                  val_set.label_idx, batch_size=64,
                                  preprocessing_fn=audio_norm)

    history = model.fit_generator(train_generator, callbacks=callbacks_list, validation_data=val_generator,
                                  epochs=config.max_epochs, use_multiprocessing=True, workers=6, max_queue_size=20)

    model.load_weights('../model/best1d_%d.h5'%i)

    # Save train predictions
    train_generator = DataGenerator(config, '../data/audio_train/', train.index, batch_size=128,
                                    preprocessing_fn=audio_norm)
    predictions = model.predict_generator(train_generator, use_multiprocessing=True, 
                                          workers=6, max_queue_size=20, verbose=1)
    np.save(PREDICTION_FOLDER + "/train_predictions_%d.npy"%i, predictions)

    # Save test predictions
    test_generator = DataGenerator(config, '../data/audio_test/', test.index, batch_size=128,
                                    preprocessing_fn=audio_norm)
    predictions = model.predict_generator(test_generator, use_multiprocessing=True, 
                                          workers=6, max_queue_size=20, verbose=1)
    np.save(PREDICTION_FOLDER + "/test_predictions_%d.npy"%i, predictions)
    K.clear_session()

/home/kownse/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:553: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=2.
  % (min_labels, self.n_folds)), Warning)


Fold:  0
##################################################
Epoch 1/1
1/1 [==============================] - 8s 8s/step - loss: 3.7359 - acc: 0.0732 - val_loss: 3.7507 - val_acc: 0.0678

Epoch 00001: val_acc improved from -inf to 0.06780, saving model to ../model/best1d_0.h5
74/74 [==============================] - 92s 1s/step
Fold:  1
##################################################
Epoch 1/1
1/1 [==============================] - 8s 8s/step - loss: 3.7067 - acc: 0.0508 - val_loss: 3.6903 - val_acc: 0.0732

Epoch 00001: val_acc improved from -inf to 0.07317, saving model to ../model/best1d_1.h5


Process ForkPoolWorker-63:
Process ForkPoolWorker-65:
Process ForkPoolWorker-64:
Process ForkPoolWorker-62:
Process ForkPoolWorker-66:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/kownse/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/kownse/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/kownse/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/kownse/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/kownse/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/kownse/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **sel

KeyboardInterrupt: 

In [5]:
pred_list = []
for i in range(1):
    pred_list.append(np.load("predictions_1d_conv/test_predictions_%d.npy"%i))
prediction = np.ones_like(pred_list[0])
for pred in pred_list:
    prediction = prediction*pred
prediction = prediction**(1./len(pred_list))
# Make a submission file
top_3 = np.array(LABELS)[np.argsort(-prediction, axis=1)[:, :3]]
predicted_labels = [' '.join(list(x)) for x in top_3]
test = pd.read_csv('../data/sample_submission.csv')
test['label'] = predicted_labels
print(test[['fname', 'label']].shape)

(9400, 2)


In [6]:
submit = test[['fname', 'label']]

In [9]:
submit.head()

,fname,label
0,00063640.wav,Hi-hat Violin_or_fiddle Fart
1,0013a1db.wav,Hi-hat Violin_or_fiddle Fart
2,002bb878.wav,Hi-hat Violin_or_fiddle Fart
3,002d392d.wav,Hi-hat Violin_or_fiddle Fart
4,00326aa9.wav,Hi-hat Violin_or_fiddle Fart


In [8]:
kaggle_util.save_result(submit,
                           "../result/1d_conv.csv",
                           'freesound-audio-tagging',
                           send = not DEBUG, index=False)

save result
